In [48]:
!pip install transformers --quiet

In [49]:
!pip install datasets --quiet

In [50]:
!wget https://raw.githubusercontent.com/banglanlp/bnlp-resources/main/sentiment/ABSA_datasets/BASA_cricket_train.tsv


--2022-01-01 19:42:46--  https://raw.githubusercontent.com/banglanlp/bnlp-resources/main/sentiment/ABSA_datasets/BASA_cricket_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 364313 (356K) [text/plain]
Saving to: ‘BASA_cricket_train.tsv.1’

BASA_cricket_train. 100%[===================>] 355.77K  --.-KB/s    in 0.03s   

2022-01-01 19:42:46 (10.3 MB/s) - ‘BASA_cricket_train.tsv.1’ saved [364313/364313]



In [51]:
!wget https://raw.githubusercontent.com/banglanlp/bnlp-resources/main/sentiment/ABSA_datasets/BASA_cricket_test.tsv

--2022-01-01 19:42:47--  https://raw.githubusercontent.com/banglanlp/bnlp-resources/main/sentiment/ABSA_datasets/BASA_cricket_test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66936 (65K) [text/plain]
Saving to: ‘BASA_cricket_test.tsv.1’

BASA_cricket_test.t 100%[===================>]  65.37K  --.-KB/s    in 0.01s   

2022-01-01 19:42:47 (5.90 MB/s) - ‘BASA_cricket_test.tsv.1’ saved [66936/66936]



In [52]:
import pandas as pd

train_df = pd.read_csv("BASA_cricket_train.tsv", sep = "\t")
test_df = pd.read_csv("BASA_cricket_test.tsv", sep = "\t")

In [53]:
train_df.tail(10)

,id,text,class_label
2075,1176,আমাদের ব্যাটসম্যানরা বাজে শটস খেলে আউট হয়েছে। ...,positive
2076,1511,সময় এসেছে রাজাকার পিচ কিউরেটর কে পাছায় লাথি দি...,negative
2077,2463,ক্রিকেটারদের প্রতিযোগিতামূলক ক্রিকেট সম্বন্ধে ...,negative
2078,1038,"দল জেতায় মুশি ক্রেডিট ম্যাসের,দল জেতায় রিয়াদ আ...",negative
2079,2911,তাদের স্পীন এটাক যে আমাদের থেকে ভালো তা কি এই ...,negative
2080,1137,বিদেশী ৫ জন এক ম্যাচে ঠিকই আছে।,neutral
2081,511,দ্রুত তাকে জাতীয়দলে সুযোগ দিয়ে ক্যারিয়ার ধংস্ব...,negative
2082,2611,"ওই নিম্ন মানের পাড়ার ক্রিকেট, ঘরোয়া লীগ, বিপিএ...",negative
2083,2613,নিলয় সবজায়গায় দেখি মাশরাফি নিয়ে বাজে কমেন্ট কর...,negative
2084,908,দ্বিতীয় ইনিংসে লিটন দাসের ট্রিপল সেঞ্চুরি চাই,positive


In [54]:
len(train_df)

2085

In [55]:
train_df["class_label"].value_counts()

negative    1515
positive     376
neutral      194
Name: class_label, dtype: int64

In [56]:
def class_to_numeric(df_column):
  if df_column == "neutral":
    return 0
  elif df_column == "positive":
    return 1
  elif df_column == "negative":
    return 2

In [57]:
train_labels = pd.Series(train_df["class_label"].apply(class_to_numeric))

In [58]:
len(train_labels) ==  len(train_df)

True

In [59]:
train_df.loc[1]

id                                                           955
text           মমিনুল হক আর মুশফিক দুজনেই নিজেদের উইকেট জঘন্য...
class_label                                             positive
Name: 1, dtype: object

In [60]:
train_df["labels"] = train_labels
train_df.head()

,id,text,class_label,labels
0,2760,আফতাব আপনি ভালো আছেন?,neutral,0
1,955,মমিনুল হক আর মুশফিক দুজনেই নিজেদের উইকেট জঘন্য...,positive,1
2,43,মিরপুর এর পিচ এ যে কেও বোলিং করতে পারে,neutral,0
3,2211,মুদ্রার ঐ পিঠ দেখা হয়ে গেল বাংলাদেশের.দেশের মা...,negative,2
4,2490,শুভকামনা রাজ্জাক ভাই।,negative,2


In [61]:
test_labels = pd.Series(test_df["class_label"].apply(class_to_numeric))
test_df["labels"] = test_labels
test_df.head()

,id,text,class_label,labels
0,2754,ইমরুল বাদে বাকি তিনজনের আউট মেনেনিতে পারছিনা ?,negative,2
1,2746,"আফতাব আহমেদ, আপনার সেই ৬টা এখনো ভুলতে পারিনি।",negative,2
2,21,জরিমানা করা হউক। ৩ মাসের বেতন কর্তন।,negative,2
3,2143,মামুর পোলারা কোচের বদ নাম করো এবার কোচ কি জিনি...,negative,2
4,426,হাথুরে বেটা কত্তটা যে খারাপ সিদ্ধান্ত বাংলাদেশ...,negative,2


In [62]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
train_dataset

Dataset({
    features: ['id', 'text', 'class_label', 'labels'],
    num_rows: 2085
})

In [63]:
test_dataset = Dataset.from_pandas(test_df)
test_dataset

Dataset({
    features: ['id', 'text', 'class_label', 'labels'],
    num_rows: 380
})

In [64]:
from transformers import AutoTokenizer 
checkpoint = "bert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [65]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer)

In [66]:
def tokenize_function(example):
  return tokenizer(
      example["text"],
      truncation = True
  )

In [67]:
tokenized_train_dataset = train_dataset.map(tokenize_function, batched = True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched = True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [68]:
tokenized_train_dataset.features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'class_label': Value(dtype='string', id=None),
 'id': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'labels': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [69]:
def process_tokenize(tokenized_dataset):
  tokenized_dataset = tokenized_dataset.remove_columns(["id","text","class_label"])
  tokenized_dataset = tokenized_dataset.with_format("torch")
  return tokenized_dataset

In [70]:
tokenized_train_dataset = process_tokenize(tokenized_train_dataset)
tokenized_train_dataset

Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
    num_rows: 2085
})

In [71]:
tokenized_test_dataset = process_tokenize(tokenized_test_dataset)
tokenized_test_dataset

Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
    num_rows: 380
})

In [72]:
tokenized_test_dataset[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'input_ids': tensor([  101,   638, 28558, 28560, 28569, 28561,   654, 28566, 28553, 28570,
           654, 28566, 28544, 28567,   648, 28567, 28555, 28549, 28555, 28570,
         28560,   100,   655, 28570, 28555, 28570, 28555, 28567, 28551, 28570,
           653, 28566, 28560, 28548, 28567, 28555, 28566,   136,   102]),
 'labels': tensor(2),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [73]:
tokenized_test_dataset[0]['attention_mask'].shape

torch.Size([39])

In [74]:
tokenized_test_dataset[1]["attention_mask"].shape

torch.Size([30])

In [75]:
tokenized_test_dataset.features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'labels': Value(dtype='int64', id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [76]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_train_dataset,batch_size = 8, shuffle = True, collate_fn = data_collator)
eval_dataloader = DataLoader(tokenized_test_dataset,batch_size = 8, collate_fn = data_collator)

In [77]:
train_dataloader

In [78]:
len(train_dataloader)

261

In [79]:
for batch in eval_dataloader:
  print(batch)
  break

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1,

In [80]:
for batch in train_dataloader:
  print(batch.keys())
  break

dict_keys(['attention_mask', 'input_ids', 'labels', 'token_type_ids'])


In [81]:
for batch in train_dataloader:
  print({k: v.shape for k  , v in batch.items() })
  break

{'attention_mask': torch.Size([8, 65]), 'input_ids': torch.Size([8, 65]), 'labels': torch.Size([8]), 'token_type_ids': torch.Size([8, 65])}


In [82]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 3)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [83]:
outputs = model(**batch)
print(outputs)

SequenceClassifierOutput(loss=tensor(1.0513, grad_fn=<NllLossBackward0>), logits=tensor([[-0.2173,  0.0005,  0.0173],
        [-0.2192, -0.0203,  0.0257],
        [-0.1948, -0.0082,  0.0582],
        [-0.1697, -0.0296, -0.0527],
        [-0.2521, -0.0589,  0.0710],
        [-0.2007, -0.0504, -0.0537],
        [-0.2036, -0.1144, -0.0721],
        [-0.2062, -0.0379, -0.0490]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [84]:
outputs.loss , outputs.logits.shape

(tensor(1.0513, grad_fn=<NllLossBackward0>), torch.Size([8, 3]))

In [85]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr = 5e-5)

In [86]:
from transformers import get_scheduler

num_epochs = 4
num_training_steps = num_epochs * len(train_dataset)
lr_scheduler = get_scheduler(
    "linear",
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps,
)
print(num_training_steps)

8340


In [87]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [88]:
batch.keys()

dict_keys(['attention_mask', 'input_ids', 'labels', 'token_type_ids'])

In [89]:
import numpy as np

def eval_model(model, data_loader, n_examples):
  model = model.eval()
  #losses = []
  correct_predictions = 0

  
  for d in data_loader:
     d = {k: v.to(device) for k, v in d.items()}
     targets = d["labels"]

     with torch.no_grad():

      outputs = model(**d)
     logits = outputs.logits
     preds = torch.argmax(logits, dim=-1)

     #loss = outputs.loss
     

     correct_predictions += torch.sum(preds == targets)
     #losses.append(loss.item())

  return correct_predictions.double() / n_examples#, np.mean(losses)


In [90]:
len(eval_dataloader)

48

In [91]:

eval_model(model,eval_dataloader,len(eval_dataloader)*8)

tensor(0.6406, device='cuda:0', dtype=torch.float64)

In [92]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()

for epoch in range(num_epochs):
  for batch in train_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  0%|          | 0/8340 [00:00<?, ?it/s]

In [93]:
eval_model(model,eval_dataloader,len(eval_dataloader)*8)

tensor(0.7109, device='cuda:0', dtype=torch.float64)